In [3]:
# import the necessary packages
# the imutils package is a collection of convenience functions
# to make basic image processing functions such as translation, rotation, resizing, skeletonization, and displaying Matplotlib images easier with OpenCV and both Python 2.7 and Python 3
import skimage.io as io
from imutils import paths
from skimage.feature import local_binary_pattern
from skimage.feature import hog
import numpy as np
from skimage import exposure
from skimage import feature
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import argparse
import cv2
import csv
import os
# for io.imshow we need to import it from skimage
# the syntax is import skimage.io as io
# linearsvc is a linear support vector machine and needs to be imported from sklearn.svm
# the syntax is from sklearn.svm import LinearSVC
# labelencoder is a label encoder and needs to be imported from sklearn.preprocessing
# the syntax is from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
# import sklearn.svm.SVC
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
# for pickle we need to import it from sklearn.externals
# the syntax for pickle is import pickle
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
np.set_printoptions(threshold=sys.maxsize)

In [12]:
# use hog implemented in skimage.feature

def get_hog():
    # get images from images array, convert to grayscale
    # and resize to 200x200 

    # try:
    #     os.remove("HOG_PCA_FEATURES.csv")
    # except OSError:
    #     pass  

    # features_file = open("HOG_PCA_FEATURES.csv", "w", newline='')
    labels = []
    pca_features = []


    for imagePath in os.listdir("Image-Segmentation"):
        # extract the label from the image path
        # we can get the label from the image name as the label is the first char of the image name
        # for example, the label of image 1_1.jpg is 1
        labels.append((int)(imagePath.split(os.path.sep)[-1][0]))
        
        # load the image, convert it to grayscale, and detect
        # edges in it
        image = cv2.imread("Image-Segmentation/" + imagePath)
        
        # extract Histogram of Oriented Gradients from the
        # test image
        # display the original image
        # if the image exists, display it and count the number of images
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (461, 260))
            (H, hogImage) = feature.hog(image, orientations=9,  pixels_per_cell=(32, 32), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualize=True, feature_vector=True)
            # H = H.reshape(-1, 1)
            # pca = PCA(0.8).fit(H)
            # components = pca.transform(H)
            pca_features.append(H)

            

            # store the label and features in the csv file
            #csv.writer(features_file).writerow([labels[-1], pca_features])

    return pca_features, np.array(labels)        


pca_features, labels = get_hog()

In [ ]:
# print("none", n)
# print("nont none", nn)
# print("none_w", n_w)
# print("nont none_w", nn_w)
# print(len(pca_features))
# print( len(os.listdir("Image-Segmentation2/MEN")))
#pca_features = np.squeeze(np.array(pca_features[:-1]), axis = 2) # remove the third dimension

In [ ]:
# temp = np.array(pca_features)
# temp = np.squeeze(temp)
# print(temp[0])
#array = pca_features.flat(1)
# instead of pca_features to be array of arrays, we want it to be a 2d array



In [ ]:
# print(pca_features[0])
# print(pca_features.shape)

#pca_features = pca_features[:-1]


In [16]:
# train the model
accuracy = 0
model = SVC(kernel='rbf', gamma=0.1, C=10)
labels_encoded = LabelEncoder().fit_transform(labels)
# partition the data into training and testing splits, using 75%
trainingData, testData, trainingLabels, testLabels = train_test_split(pca_features, labels_encoded, test_size=0.2, random_state=42)
trainingData, validationData, trainingLabels, validationLabels = train_test_split(trainingData, trainingLabels, test_size=0.2, random_state=42)
trainingLabels = np.reshape(trainingLabels, trainingLabels.shape[0])

def train_model_hog():
    
    #model = SVC(random_state=42) # ramdom state is used to get the same results each time. It means that the model will be trained on the same data each time
     # rbf means that the model will use the gaussian kernel as it stands for radial basis function

    

    print("[INFO] evaluating...", flush=True)
    
    # train the model
    # how to continue to train SVM based on the previous model
    # this is done by using the partial_fit method
    # syntax: model.partial_fit(trainingData, trainingLabels)
    
    
    # trainingData = trainingData.reshape(-1, 1)
    # trainingLabels = trainingLabels.reshape(-1, 1)
    # now padding the data to be 2D array
    
    #print(trainingLabels)
    model.fit(trainingData, trainingLabels)

    # evaluate the model and update the accuracies list
    acc = model.score(validationData, validationLabels)
    accuracy = acc
    print("[INFO] accuracy: {:.2f}%".format(acc * 100), flush=True)
    # dump the classifier to file
    print("[INFO] dumping classifier to file...", flush=True)
    f = open("SVM_classifier.pkl", "wb") # wb = write binary
    f.write(pickle.dumps(model))
    f.close()


train_model_hog()
print(accuracy)

[INFO] evaluating...
[INFO] accuracy: 72.26%
[INFO] dumping classifier to file...
0


In [17]:
# evaluate the model and update the accuracies list
acc = model.score(testData, testLabels)
accuracy = acc
print("[INFO] accuracy: {:.2f}%".format(acc * 100), flush=True)
# dump the classifier to file
print("[INFO] dumping classifier to file...", flush=True)
f = open("SVM_classifier.pkl", "wb") # wb = write binary
f.write(pickle.dumps(model))
f.close()

[INFO] accuracy: 72.33%
[INFO] dumping classifier to file...


In [ ]:
# # train the model
# def train_model_hog():
#     k = 0
#     #model = SVC(random_state=42) # ramdom state is used to get the same results each time. It means that the model will be trained on the same data each time
#     model = SVC(kernel='rbf', gamma=0.01, C=10) # rbf means that the model will use the gaussian kernel as it stands for radial basis function
  
#     labels_data = pd.read_csv("HOG_PCA_FEATURES.csv", header=None, sep=',')

#     data_temp = np.array(labels_data)
#     for j in range (len(labels_data)):
#         temp = data_temp[j, 1].replace("[", "")
#         temp = temp.replace("\n", "")
#         temp = temp.replace("]", "")
#         temp= temp.split(" ")
#         temp = np.array(temp)
#         if(temp.__contains__('')):
#             temp = np.delete(temp, np.where(temp == ''))
#         data.append(np.array(temp, dtype=np.float32))
#         labels.append(labels_data.iloc[j, 0])

#     labels_data = pd.read_csv("HOG_PCA_FEATURES.csv", header=None, sep=',')

#     data_temp = np.array(labels_data)
#     for j in range (len(labels_data)):
#         temp = data_temp[j, 1].replace("[", "")
#         temp = temp.replace("\n", "")
#         temp = temp.replace("]", "")
#         temp= temp.split(" ")
#         temp = np.array(temp)
#         if(temp.__contains__('')):
#             temp = np.delete(temp, np.where(temp == ''))
#         #print(data_temp)
#         data.append(np.array(temp, dtype=np.float32))
#         labels.append(labels_data.iloc[j, 0])
    
#         data = np.array(data, dtype="float")
#         labels = LabelEncoder().fit_transform(labels)
#         # partition the data into training and testing splits, using 75%
#         trainingData, testData, trainingLabels, testLabels = train_test_split(data, labels, test_size=0.25, random_state=42)
        

#         print("[INFO] evaluating...", flush=True)
        
#         # train the model
#         # how to continue to train SVM based on the previous model
#         # this is done by using the partial_fit method
#         # syntax: model.partial_fit(trainingData, trainingLabels)
#         model.fit(trainingData, trainingLabels)

#         # evaluate the model and update the accuracies list
#         acc = model.score(testData, testLabels)
#         print("[INFO] accuracy: {:.2f}%".format(acc * 100), flush=True)
#         # dump the classifier to file
#         print("[INFO] dumping classifier to file...", flush=True)
#         f = open("SVM_classifier.pkl", "wb") # wb = write binary
#         f.write(pickle.dumps(model))
#         f.close()